Tarea 3

Descargar el archivo wiki-sub.jsonl y realizar las siguientes actividades. Este archivo tiene 100k documentos, cada uno de ellos corresponde a la introducción de un artículo de Wikipedia.

Descargar el archivo query-sub.jsonl. Este archivo contiene 50 consultas que serán utilizadas para evaluar las propuestas de RI. Cada consulta tiene los identificadores de los documentos que fueron evaluados como relevantes.

 

Actividad 1

Implementar 2 modelos de RI que utilicen los algoritmos LSA (truncated SVD) y LDA para identificar la distibución de tópicos de los documentos. Los modelos de RI deben usar esta distribución de tópicos para recuperar los documentos relevantes para una consulta dada. Existen varias formas de utilizar el análisis de tópicos en RI. Explicar cuál es la técnica utilizada y cuál es la intención de usar el análisis de tópicos en RI.

 

Actividad 2

Utilizando las consultas del archivo query-sub.jsonl, obtener las curvas recall-precisión para los 2 modelos de RI basados en tópicos. Reportar el mejor valor de F1 obtenido para cada uno. También reportar el MAP resultante.

 

Actividad 3

Reportar precisión, recall y F1@5 y F1@10, para los 2 sistemas.

 

Actividad 4

Discutir los resultados obtenidos.

In [23]:
import pandas as pd
from ast import literal_eval
import numpy as np 
from scipy.linalg import inv 

In [2]:
#Carga de los documentos en formato csv, se usa literal_eval para que la lista de tokens se guarden como lista y no como cadena
df=pd.read_csv("wiki-sub.csv")
df["lemmas"]=df["lemmas"].apply(literal_eval)
df["stems"]=df["stems"].apply(literal_eval)
df

,id,text,lemmas,stems
0,HazMat_-LRB-film-RRB-,HazMat is a 2013 horror film written and direc...,"[hazmat, 2013, horror, film, write, direct, lo...","[hazmat, 2013, horror, film, written, direct, ..."
1,Pseudodrephalys,Pseudodrephalys is a genus of South American s...,"[pseudodrephaly, genus, south, american, skipp...","[pseudodrephali, genu, south, american, skippe..."
2,Anathallis_linearifolia,Anathallis linearifolia is a species of orchid...,"[anathalli, linearifolia, species, orchid, lin...","[anath, linearifolia, speci, orchid, linearifo..."
3,John_A._McElwain,"John Allen McElwain -LRB- September 21 , 1794 ...","[john, allen, mcelwain, lrb, september, 21, 17...","[john, allen, mcelwain, lrb, septemb, 21, 1794..."
4,Usserød_Å,"Usserød Å , the principal drainage of Sjælsø L...","[usserød, å, principal, drainage, sjælsø, lake...","[usserød, å, princip, drainag, sjælsø, lake, s..."
...,...,...,...,...
99605,Lets_Raise_A_Million,The Let 's Raise a Million Project -LRB- LRAM ...,"[let, s, raise, million, project, lrb, lram, r...","[let, s, rais, million, project, lrb, lram, rr..."
99606,Kunita–Watanabe_theorem,"In stochastic calculus , the Kunita -- Watanab...","[stochastic, calculus, kunita, watanabe, theor...","[stochast, calculu, kunita, watanab, theorem, ..."
99607,Montenegrin_Amateur_Radio_Pool,The Montenegrin Amateur Radio Pool -LRB- MARP ...,"[montenegrin, amateur, radio, pool, lrb, marp,...","[montenegrin, amateur, radio, pool, lrb, marp,..."
99608,Mouth_and_foot_painting,Mouth and foot painting is a technique to crea...,"[mouth, foot, painting, technique, create, dra...","[mouth, foot, paint, techniqu, creat, draw, pa..."


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
                            max_features= 1000, # máximo número de términos
                            max_df = 0.5, 
                            smooth_idf=True)
corpus=df['lemmas'].apply(lambda x: " ".join(x))
X = vectorizer.fit_transform(corpus)
print("Tamaño de la matriz T-D: ", X.shape)

Tamaño de la matriz T-D:  (99610, 1000)


In [4]:
from sklearn.utils.extmath import randomized_svd

#Se puede usar SVD truncado con randomized_svd
U, Sigma, VT = randomized_svd(X, 
                              n_components=15,
                              n_iter=100,
                              random_state=123)

In [5]:
print(U.shape)
print(Sigma.shape)
print(VT.shape)

(99610, 15)
(15,)
(15, 1000)


In [14]:
VT

array([[ 0.0262894 ,  0.04039923,  0.01823786, ...,  0.02108627,
         0.00942668,  0.01964438],
       [-0.00975359, -0.01962459, -0.0095519 , ..., -0.01765256,
        -0.00820902, -0.01334441],
       [-0.00215708, -0.00425972, -0.0004727 , ...,  0.00156624,
        -0.00113346,  0.00457579],
       ...,
       [-0.00554148, -0.01362898, -0.01047816, ..., -0.00735678,
        -0.00349659, -0.00498807],
       [-0.00318503, -0.01936598, -0.01381321, ..., -0.0021241 ,
        -0.00134991, -0.01117282],
       [ 0.00239065,  0.01004497,  0.00974218, ..., -0.00170298,
         0.00105125, -0.00417417]])

In [15]:
Sigma

array([47.85584229, 40.90544486, 36.18944078, 33.02638267, 30.81539163,
       30.05130419, 28.85834789, 27.79169482, 25.40590642, 24.65237903,
       24.39504871, 24.04355875, 23.75263639, 23.40902848, 23.35902272])

In [31]:
data=pd.read_csv("ejemplo.csv",header=None)
data.index=["ship","boat","ocean","wood","tree"]
data

,0,1,2,3,4,5
ship,1,0,1,0,0,0
boat,0,1,0,0,0,0
ocean,1,1,0,0,0,0
wood,1,0,0,1,1,0
tree,0,0,0,1,0,1


In [11]:
U, S, VT = randomized_svd(data.values, 
                              n_components=5,
                              n_iter=100,
                              random_state=123)

In [39]:
#Reduccion de dimensionalidad
k=2
S2=np.zeros(data.shape)
S2[:k,:k]=np.diag(S[:k])
S2

array([[2.16250096, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.59438237, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [42]:
S2=np.diag(S[:k])
S2

array([[2.16250096, 0.        ],
       [0.        , 1.59438237]])

In [20]:
VT2=np.zeros(VT.shape)
VT2[:k,:]=VT[:k,:]
VT2

array([[ 0.74862305,  0.2797116 ,  0.2036288 ,  0.44656311,  0.32509596,
         0.12146715],
       [-0.28645399, -0.52845914, -0.18576119,  0.6255207 ,  0.21987976,
         0.40564094],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [43]:
VT2=VT[:k, :]
VT

array([[ 7.48623048e-01,  2.79711603e-01,  2.03628802e-01,
         4.46563110e-01,  3.25095956e-01,  1.21467154e-01],
       [-2.86453991e-01, -5.28459139e-01, -1.85761186e-01,
         6.25520701e-01,  2.19879758e-01,  4.05640944e-01],
       [-2.79711603e-01,  7.48623048e-01, -4.46563110e-01,
         2.03628802e-01, -1.21467154e-01,  3.25095956e-01],
       [ 1.08898021e-16, -4.90934527e-16, -5.77350269e-01,
         1.66217984e-16,  5.77350269e-01, -5.77350269e-01],
       [-5.28459139e-01,  2.86453991e-01,  6.25520701e-01,
         1.85761186e-01,  4.05640944e-01, -2.19879758e-01]])

In [21]:
U2=np.zeros(U.shape)
U2[:,:k]=U[:,:k]
U2

array([[ 0.44034748, -0.29617436,  0.        ,  0.        ,  0.        ],
       [ 0.12934635, -0.33145069,  0.        ,  0.        ,  0.        ],
       [ 0.47553026, -0.51111524,  0.        ,  0.        ,  0.        ],
       [ 0.70302032,  0.35057241,  0.        ,  0.        ,  0.        ],
       [ 0.26267284,  0.64674677,  0.        ,  0.        ,  0.        ]])

In [44]:
U2=U[:,:k]
U2

array([[ 0.44034748, -0.29617436],
       [ 0.12934635, -0.33145069],
       [ 0.47553026, -0.51111524],
       [ 0.70302032,  0.35057241],
       [ 0.26267284,  0.64674677]])

In [46]:
#proyectar las palabras al espacio de topicos
proy= U2 @ S2
proy = pd.DataFrame(proy, index=data.index) # Words as k-vectors
proy

,0,1
ship,0.952252,-0.472215
boat,0.279712,-0.528459
ocean,1.028335,-0.814913
wood,1.520282,0.558946
tree,0.568030,1.031162


In [48]:
#query 
myquery = np.where((data.index=='wood') | (data.index=='tree'))[0]
myquery

array([3, 4])

In [50]:
q=proy.iloc[myquery,:]
q

,0,1
wood,1.520282,0.558946
tree,0.568030,1.031162


In [52]:
r=VT2.T@q
r

,0,1
0,0.975404,0.123060
1,0.125060,-0.388583
2,0.204055,-0.077732
3,1.034217,0.894618
4,0.619136,0.408443
5,0.415081,0.486175


In [75]:
S2.T.shape

(2, 2)

In [76]:
inv(S2)@(U2@q).T

,0,1,2,3,4
0,0.231776,0.003870,0.200051,0.586323,0.354547
1,-0.037176,-0.169019,-0.163854,0.473191,0.510367


In [73]:
a=S2@U2.T
a.shape

(2, 5)

In [58]:
inv(S2)@U2@r

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 2)

In [54]:
idx = np.argsort(VT2.T @ q)[::-1]
kl = list(data.index)
keys = [kl[i] for i in idx]
docs = [data.index[k] for k in keys]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [22]:
A2= U2@S2@VT2
A2

array([[ 0.8481456 ,  0.51590232,  0.28162515,  0.12986018,  0.20574267,
        -0.07588249],
       [ 0.36077778,  0.35750764,  0.15512454, -0.20565325, -0.02526436,
        -0.18038889],
       [ 1.00327014,  0.71828543,  0.36077778, -0.05052871,  0.15512454,
        -0.20565325],
       [ 0.97800578,  0.12986018,  0.20574267,  1.0285345 ,  0.61713858,
         0.41139591],
       [ 0.12986018, -0.38604214, -0.07588249,  0.89867432,  0.41139591,
         0.4872784 ]])

In [41]:
VT2

array([[ 0.74862305,  0.2797116 ,  0.2036288 ,  0.44656311,  0.32509596,
         0.12146715],
       [-0.28645399, -0.52845914, -0.18576119,  0.6255207 ,  0.21987976,
         0.40564094],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [36]:
#query = shio and boat
q = U2[:, :k] @ np.diag(S2[:k]) #proyeccion del la consulta al espacio de topicos
q

array([ 0.48003667, -0.24874754,  0.21342152,  2.07922858,  1.59919191])

In [33]:
#consulta
resultado=[]
for col in data.columns:
    doc=data[col]
    resultado.append(np.dot(q,doc))
resultado=np.array(resultado)
resultado

array([ 2.77268678, -0.03532601,  0.48003667,  3.67842049,  2.07922858,
        1.59919191])

In [34]:
np.argmax(resultado)

3